In [1]:
%pip install pandas
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 43.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import tabula
import pandas as pd



directory = 'data/uk'

pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

for pdf_file in pdf_files:
    file_path = os.path.join(directory, pdf_file)
    tabula.convert_into(file_path, f"{directory}/{pdf_file}.csv", output_format="csv", pages='all')
    print(f"Converted {pdf_file} to csv")

Converted ufo_report_2008.pdf to csv
Converted ufo_report_2009.pdf to csv


In [14]:
us = pd.read_csv('data/us_ufo_sightings.csv')
us.shape

# france = pd.read_csv('data/france_ufo_sightings.csv', encoding='utf-8')
# france.head()

# uk = pd.concat([pd.read_csv(f"{directory}/{f}") for f in os.listdir(directory) if f.endswith('.csv')])
# uk.head()

(80332, 11)